# Prepare data to support DEM comparison - Wagga Wagga

Various DEM/DSM and elevation values are used in the workflow. 
It's important to understand consistency between the DEM values.

DEMs to be compared

* NSW Spatial Services 1m - 2020 - "GDA2020 / MGA zone 55" ("EPSG:7855"), AHD71 - using local Geoid model
* ~~AGO 1m - Tamar 2008 - "GDA94 / MGA zone 55"~~
* ~~AGO 7cm (DSM) - 2021 - "WGS 84 / UTM zone 55S" ("EPSG:32755)~~

Comparison methods to be considered

* Comparison to surveyed ground control points
* Comparison of downsampled minimum elevation
* Comparison of points over road

In [116]:
import geopandas as gpd

### Ground control points

NSW Survey Control Information Management System (SCIMS) FeatureServer: https://portal.spatial.nsw.gov.au/server/rest/services/SurveyMarkGDA2020/FeatureServer

In [117]:
# constrain using building dataset boundary
buildings_path = "/home/lng/sharepoint/Projects - Documents/Projects - Data Analytics/127 Residential Dwelling Floor Height/4 Executing/Data Exploration/overture/wagga_building.gpkg"

buildings  = gpd.read_file(buildings_path)

In [118]:
# Exported from NSW SCIMS Feature Server
gpkg_path = "/home/lng/sharepoint/Projects - Documents/Projects - Data Analytics/127 Residential Dwelling Floor Height/4 Executing/Data Exploration/Ground Control/NSW/wagga_scims_marks.gpkg"
gdf = gpd.read_file(gpkg_path)


In [119]:
# Project to GDA2020 MGA55
gdf = gdf.to_crs(7855)


In [120]:
# Inspect data
gdf.marksymbol_label.unique()

array(['SS - Established GDA2020 and Accurate AHD71',
       'SS - Established GDA2020',
       'PM - Established GDA2020 and Accurate AHD71',
       'PM - Established GDA2020', 'CP - Established GDA2020',
       'TS - Established GDA2020 and Accurate AHD71',
       'GB - Established GDA2020 and Accurate AHD71',
       'CR - Established GDA2020', 'SS - Unknown', 'PM - Unknown',
       'SS - Accurate AHD71', 'PM - Accurate AHD71',
       'CR - Approximate GDA2020 Only', 'SS - Approximate GDA2020 Only',
       'PM - Accurate AHD71 and Approximate GDA2020',
       'SS - Accurate AHD71 and Approximate GDA2020',
       'PM - Approximate GDA2020 Only'], dtype=object)

In [121]:
gdf.monumenttype.unique()

array(['UNKNOWN', 'SSM IN CONC', 'NAIL IN BITUMEN',
       'STAR PICKET IN C/BOX', 'S/S PIN IN C/BOX', 'CONC PILLAR',
       'BRASS PLAQUE', 'PLAQUE', 'DEEP DRIVEN ROD', 'STEEL PILLAR',
       'S/S PIN IN CONC', 'GI PIPE IN SOIL', 'SSM IN ROCK',
       'ANTENNA REFERENCE', 'REFERENCE TREE', 'DH & WING IN KERB',
       'FENCE POST', 'GI PIPE', 'PEG', 'CONC BLOCK', 'DH & WING',
       'IRON SPIKE', 'PHOTO POINT', 'SSM TYPE 1', 'BROAD ARROW',
       'ALIGNMENT PIN', 'STAR PICKET', 'SSM TYPE 15', 'SSM TYPE 16',
       'SSM TYPE 2', 'SSM'], dtype=object)

Position quality criteria
* markstatus ="F" (found), "R" (restricted)  and
* gdaclass in "3A", "2A", "A", "B", "C", "D", "E" (established survey marks)
* ahdclass in "L2A", "LA", "LB", "LC", "LD", "LE", "2A", "A", "B" (accurate heights)


In [122]:
# Filter using above rules
filter = gdf.markstatus.isin(["F", "R"])
filter &= gdf.gdaclass.isin(["3A", "2A", "A", "B", "C", "D"])
filter &= gdf.ahdclass.isin(["L2A", "LA", "LB", "LC", "LD", "LE", "2A", "A", "B"])
gdf_filtered = gdf[filter]
gdf_filtered.monumenttype.unique()


array(['UNKNOWN', 'SSM IN CONC', 'CONC PILLAR', 'S/S PIN IN C/BOX',
       'STAR PICKET IN C/BOX', 'PLAQUE', 'STEEL PILLAR',
       'S/S PIN IN CONC', 'BRASS PLAQUE', 'SSM IN ROCK',
       'DEEP DRIVEN ROD', 'ANTENNA REFERENCE'], dtype=object)

In [123]:
len(gdf_filtered)

206

Mark location criteria

* unknown
* c/box (below ground)
* antenna (above ground)
* pillar (above ground)


In [124]:
# Filter out unreliable, not at ground level
filter = ["unknown", "c/box", "antenna", "pillar"]

gdf_filtered = gdf_filtered[~gdf_filtered.monumenttype.str.lower().str.contains('|'.join(filter))]

len(gdf_filtered)

86

In [131]:
gdf_filtered.marksymbol_label.unique()

array(['SS - Established GDA2020 and Accurate AHD71',
       'PM - Established GDA2020 and Accurate AHD71',
       'GB - Established GDA2020 and Accurate AHD71'], dtype=object)

In [125]:
gdf_filtered.monumenttype.unique()

array(['SSM IN CONC', 'PLAQUE', 'S/S PIN IN CONC', 'BRASS PLAQUE',
       'SSM IN ROCK', 'DEEP DRIVEN ROD'], dtype=object)

In [126]:
gdf_filtered.monumenttype.describe()

count              86
unique              6
top       SSM IN CONC
freq               77
Name: monumenttype, dtype: object

In [129]:
gdf_filtered.ahdheight_label = gdf_filtered.ahdheight_label.astype(float)

In [130]:
# Save as GeoPackage
gdf_filtered.to_file("/home/lng/sharepoint/Projects - Documents/Projects - Data Analytics/127 Residential Dwelling Floor Height/4 Executing/Data Exploration/Ground Control/NSW/wagga_scims_marks_filtered.gpkg")